In [8]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests
r = requests.get('https://www.comedera.com/recetas-por-paises/')
content=r.text
print(content)

<!DOCTYPE html><html lang="es"><head><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="robots" content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" /><link media="all" href="https://www.comedera.com/wp-content/cache/autoptimize/css/autoptimize_4824d269ea4aeec6a218e0891d101954.css" rel="stylesheet"><title>Recetas por países - Comedera.Com</title><meta name="description" content="Gastronomía del mundo. Recetas españolas, italianas, mexicanas, chinas, peruanas, venezolanas y más. Platos para hacer en casa y sorprender a tus invitados." /><link rel="canonical" href="https://www.comedera.com/recetas-por-paises/" /><link rel="next" href="https://www.comedera.com/recetas-por-paises/page/2/" /><meta property="og:locale" content="es_ES" /><meta property="og:type" content="article" /><meta property="og:title" content="Recetas por países - Comedera.Com" /><meta property="og:description" content="Gastronomía 

In [11]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(content, 'lxml')
formatted=soup.prettify()

# fichier="contenido.html"
# with open(fichier, "w") as file:
#     file.write(formatted)


In [12]:
div1 = soup.find(
    "div",
    attrs={"class": "site-container"}).find(
        "nav",
        attrs={"aria-label":"Secundario", "class":"nav-secondary"}).find(
            "div",
            attrs={"class":"wrap"}).find(
                "li",
                attrs={"class":"menu-item menu-item-type-taxonomy menu-item-object-category current-menu-item menu-item-has-children menu-item-20277", "id":"menu-item-20277"}).find(
                    "ul",
                    attrs={"class":"sub-menu"})
            

#print(div1.prettify())
elem_a = div1.find_all("a")
enlaces=[]
isin=["peruana/", "venezolana/", "colombiana/", "ecuatoriana/", "salvadorena/", "hondurena/"]
for a in elem_a:
    href = a.get("href")
    if href and any(href.endswith(word) for word in isin):
        enlaces.append(href)
print(enlaces)


['https://www.comedera.com/recetas-comida-peruana/', 'https://www.comedera.com/comida-venezolana/', 'https://www.comedera.com/comida-colombiana/', 'https://www.comedera.com/comida-ecuatoriana/', 'https://www.comedera.com/comida-salvadorena/', 'https://www.comedera.com/comida-hondurena/']


In [28]:
toutes_recettes=[]
id_r = 1
for p_index, enlace in enumerate(enlaces, start=1):
    #print(p_index, enlace)
#solo para intentar
#enlace="https://www.comedera.com/recetas-comida-peruana/"
    r2 = requests.get(enlace)
    if r2.status_code == 200:
        soup2 = BeautifulSoup(r2.text, 'html.parser')
        formateado=soup2.prettify()
   
    #je garde mes contenus html dans des fichiers parce qu'il est plus facile de reconnaître la ligne que je dois extraire comme ça
        # nom_fichier= f"contenido_{index}.html"
    
        # with open (nom_fichier, 'w', encoding="utf-8") as file:
        #     file.write(formateado)
            
        recetas_pays =[]
        div2=soup2.find(
            "div",
            attrs={"class": "site-container"}).find(
                "div",
                attrs={"class":"site-inner"}).find(
                    "div",
                    attrs={"class":"content-sidebar-wrap"}).find(
                        "main",
                        attrs={"class":"content", "id":"genesis-content"})
        #changer la quantité ici:              
        max_links = 5
        articles = div2.find_all("article")
        links_recettes = [art.find('a').get("href") for art in articles[:max_links]]
       
        
        for r_index, link in enumerate(links_recettes, start=1):
            #print(r_index,link)
            r_receta = requests.get(link)
            if r_receta.status_code == 200:
                soup3 = BeautifulSoup(r_receta.text, 'html.parser')
                formateado = soup3.prettify()
                nom_fichier = f"receta_{p_index}_{r_index}.html"

                # with open(nom_fichier, 'w', encoding="utf-8") as file:
                #     file.write(formateado)
                nom_recette=soup3.head.find("title").text
                nom_r=nom_recette.split("-")[0]
                #print(nom_r)
                
                div_i=soup3.find(
                    "div", 
                    attrs={"class":"wprm-recipe-ingredient-group"})
                ingredient_items=div_i.find_all("li")
                ingredients_r=[]
                for ingredient in ingredient_items:
                    ingredients_r.append(ingredient.text.strip())
                
                div_p=soup3.find(
                    "div", 
                    attrs={"class":"wprm-recipe-instruction-group"})
                pas_items=div_p.find_all("li")
                pas_r=[]
                for pas in pas_items:
                    pas_r.append(pas.text.strip())
                
                info_recettes = {
                    "id":id_r,
                    "nom":nom_r,
                    "link": link,
                    "ingredients": ingredients_r,
                    "pas": pas_r           
                }
                toutes_recettes.append(info_recettes)
                id_r += 1
print(toutes_recettes)
                
                
                
            
        

[{'id': 1, 'nom': 'Receta para hacer arroz chaufa peruano ', 'link': 'https://www.comedera.com/receta-para-hacer-arroz-chaufa/', 'ingredients': ['1/2 kg de pechuga', '1/2 kg de arroz', '1 taza  de cebolla de verdeo  cortada', '2 dientes de ajo', '2 cditas de jengibre  rallado', '3/4 taza de salsa de soja', '1 cdita de condimento 5 especias', '1/2 cdita de pimienta', 'Sal al gusto', '6 huevos'], 'pas': ['Inicia la receta y corta la pechuga en cubos medianos, coloca en un bol.', 'Posteriormente, toma los dientes de ajo y filetea e incorpora la mitad al bol con el pollo. También agrega el condimento 5 especias, el jengibre, un poco de pimienta y la mitad de la salsa de soja.', 'Mezcla muy bien el pollo con los condimentos. Deja marinar por unos minutos.', 'Cocina el arroz con agua y sal hasta que quede al dente y suelto, apaga y deja enfriar.', 'Calienta un sartén con un poco de aceite, mientras vierte los huevos en un bol, bate y posteriormente incorpora en el sartén. Cocina hasta hacerl

In [29]:
import pandas as pd

df_recettes = pd.DataFrame(toutes_recettes)
df_recettes.to_csv("recettes_1.csv", index=False)
print(df_recettes.head())

   id                                       nom  \
0   1   Receta para hacer arroz chaufa peruano    
1   2    Receta fácil de sopa de pollo peruana    
2   3  Receta de tiradito de salmón con sésamo    
3   4          Receta de arroz chaufa de carne    
4   5          Chaufa de pollo, receta peruana    

                                                link  \
0  https://www.comedera.com/receta-para-hacer-arr...   
1  https://www.comedera.com/haz-sopa-de-pollo-per...   
2  https://www.comedera.com/tiradito-de-salmon-co...   
3  https://www.comedera.com/receta-de-arroz-chauf...   
4  https://www.comedera.com/chaufa-de-pollo-recet...   

                                         ingredients  \
0  [1/2 kg de pechuga, 1/2 kg de arroz, 1 taza  d...   
1  [1 pollo completo despresado, 1 trozo pequeño ...   
2  [1  cebolla  picada, 1 diente de ajo, 1 cda az...   
3  [14,10  onzas (400 gr) de carne, 2 tbps (Cucha...   
4  [4 cups (tazas) de arroz cocido suelto, 7 oz (...   

                   